In [7]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.chrome.options import Options


In [8]:
options = Options()
options.add_argument('--no-sandbox') # Bypass OS security model

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False, chrome_options=options)

In [9]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)